# Data Ingestion, Cleaning & Transformation

In [1]:
# Import the necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Create the input folder for the raw data using os
os.makedirs('raw_data', exist_ok=True)


In [3]:
# Ingest the data
df = pd.read_csv('raw_data/train.csv')

# View the shape of the data
df.shape

(1460, 81)

In [4]:
# Review the information in the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
# Show the features in the data that have missing values and the number of missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

# Get the data types of the features with missing values
missing_values_data_types = df.dtypes[missing_values.index]

# Combine the missing values and data types into a single DataFrame
missing_values_info = pd.DataFrame({'Missing Values': missing_values, 'Data Type': missing_values_data_types})

print(missing_values_info)


              Missing Values Data Type
LotFrontage              259   float64
Alley                   1369    object
MasVnrType               872    object
MasVnrArea                 8   float64
BsmtQual                  37    object
BsmtCond                  37    object
BsmtExposure              38    object
BsmtFinType1              37    object
BsmtFinType2              38    object
Electrical                 1    object
FireplaceQu              690    object
GarageType                81    object
GarageYrBlt               81   float64
GarageFinish              81    object
GarageQual                81    object
GarageCond                81    object
PoolQC                  1453    object
Fence                   1179    object
MiscFeature             1406    object


In [6]:
# List of columns to analyze
columns = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

# Loop through each column and calculate skewness
for column in columns:
    # Drop missing values and calculate skewness
    skewness = df[column].dropna().skew()
    # Print the result
    print(f'The distribution of the {column} feature is skewed by {skewness:.2f}')


The distribution of the LotFrontage feature is skewed by 2.16
The distribution of the MasVnrArea feature is skewed by 2.67
The distribution of the GarageYrBlt feature is skewed by -0.65


As the distribution of the three numeric variables with missing values are skewed, those values will be replaced with the median value, rather than then mean value.

In [7]:
# List of columns to fill missing values
columns = ['MasVnrArea', 'GarageYrBlt', 'LotFrontage']

# Loop through each column and fill missing values with the median
for column in columns:
    df[column] = df[column].fillna(df[column].median())


In [8]:
# Show the features in the data that have missing values and the number of missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

# Get the data types of the features with missing values
missing_values_data_types = df.dtypes[missing_values.index]

# Combine the missing values and data types into a single DataFrame
missing_values_info = pd.DataFrame({'Missing Values': missing_values, 'Data Type': missing_values_data_types})

print(missing_values_info)

              Missing Values Data Type
Alley                   1369    object
MasVnrType               872    object
BsmtQual                  37    object
BsmtCond                  37    object
BsmtExposure              38    object
BsmtFinType1              37    object
BsmtFinType2              38    object
Electrical                 1    object
FireplaceQu              690    object
GarageType                81    object
GarageFinish              81    object
GarageQual                81    object
GarageCond                81    object
PoolQC                  1453    object
Fence                   1179    object
MiscFeature             1406    object


In [9]:
# Define the columns to replace missing values with 'n/a'
columns_to_replace = [
    'LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 
    'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu', 
    'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 
    'PoolQC', 'Fence', 'MiscFeature'
]

# Replace NaN values with 'n/a'
df[columns_to_replace] = df[columns_to_replace].fillna('Not Applicable')


In [10]:
# Create a feature called InternalSF
df['InternalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']

In [11]:
# Create PorchDeckSF feature
df['PorchDeckSF'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch'] + df['WoodDeckSF']

In [12]:
# Step 1: Compute the median SalePrice per Neighborhood
neighborhood_prices = df.groupby('Neighborhood')['SalePrice'].median()

# Step 2: Calculate percentiles based on SalePrice
percentiles = neighborhood_prices.rank(pct=True)

# Step 3: Assign bucket levels
def assign_bucket(pct):
    if pct < 0.25:
        return 'Level_1'
    elif pct < 0.50:
        return 'Level_2'
    elif pct < 0.75:
        return 'Level_3'
    else:
        return 'Level_4'

# Step 4: Create a mapping dictionary
neighborhood_price_buckets = percentiles.apply(assign_bucket)

# Step 5: Merge back into the original DataFrame
df['Neighborhood_Price_Bucket'] = df['Neighborhood'].map(neighborhood_price_buckets)

In [13]:
# To be used when creating the data for the test
# Convert to DataFrame for better readability
neighborhoods_per_level_df = (
    df[['Neighborhood', 'Neighborhood_Price_Bucket']]
    .drop_duplicates()
    .groupby('Neighborhood_Price_Bucket')['Neighborhood']
    .apply(list)
    .reset_index()
)

# Display the result
print(neighborhoods_per_level_df)

  Neighborhood_Price_Bucket                                       Neighborhood
0                   Level_1  [OldTown, BrkSide, IDOTRR, MeadowV, Edwards, B...
1                   Level_2  [Mitchel, Sawyer, NAmes, NPkVill, SWISU, Blueste]
2                   Level_3  [CollgCr, NWAmes, SawyerW, Gilbert, ClearCr, B...
3                   Level_4  [Veenker, Crawfor, NoRidge, Somerst, NridgHt, ...


In [14]:
# Make a copy of the data
df_copy = df.copy()

In [15]:
# Create a subfolder called outputs using os module
os.makedirs('outputs', exist_ok=True)


In [16]:
# Create a cv of the cleaned data
df_copy.to_csv('outputs/train_cleaned.csv', index=False)